In [1]:
import json
import random
import pandas as pd

In [3]:
name = "yelp"

inter_path = f"../../../data/resource/{name}/{name}.inter"
user_path = f"../../../data/resource/{name}/{name}.user"
item_path = f"../../../data/resource/{name}/{name}.item"

mini_inter_path = f"../../../data/resource/mini_{name}/{name}.inter"
mini_user_path = f"../../../data/resource/mini_{name}/{name}.user"
mini_item_path = f"../../../data/resource/mini_{name}/{name}.item"

w_pt_path = f"./data/llm/{name}/{name}_pt.json"
w_ft_path = f"./data/llm/{name}/{name}_ft.json"
w_pd_path = f"./data/llm/{name}/{name}_pd.json"

In [5]:
# 读数据
inter_data = pd.read_csv(inter_path, sep="\t").sort_values(by=['user_id:token','rating:float', 'timestamp:float'])
user_data = pd.read_csv(user_path, sep="\t").sort_values(by=['user_id:token'])


In [6]:
item_data = pd.read_csv(item_path, sep="\t").sort_values(by=['item_id:token'])

In [9]:
def build_item_other(row):
    df1 = inter_data[inter_data['item_id:token'] == row['item_id:token']]
    total = df1.shape[0]
    likes_count = df1[df1['rating:float'] > threshold].shape[0]
    # dislikes_count = total - likes_count
    liking_rate = round(likes_count/total, 4)
    return {'liking_rate':liking_rate}

In [11]:
data_u = inter_data
data_u = data_u.groupby(['user_id:token']).count()
data_u["count"] = data_u["item_id:token"]
data_u = data_u[data_u["count"] >= 50]
mini_data_u = data_u.reset_index()
mini_data_u

,user_id:token,item_id:token,rating:float,timestamp:float,useful:float,funny:float,cool:float,review_id:token,count
0,-3s52C4zL_DHRK0ULG6qtg,87,87,87,87,87,87,87,87
1,-7XrSrJfHndHc_taEXurTw,56,56,56,56,56,56,56,56
2,-B-QEUESGWHPE_889WJaeg,117,117,117,117,117,117,117,117
3,-FxsSuwDbIII7yo5BjHpiA,114,114,114,114,114,114,114,114
4,-G7Zkl1wIWBBmD0KRy_sCw,370,370,370,370,370,370,370,370
...,...,...,...,...,...,...,...,...,...
1582,zsXoPyTcU8ThZGbtAB-Vug,51,51,51,51,51,51,51,51
1583,zu-e06_BM_TdkAZEKMrIww,68,68,68,68,68,68,68,68
1584,zv7tpu7xeaNyAeFG03d2CA,66,66,66,66,66,66,66,66
1585,zwXmvn1op5LuFF2Kveqaug,54,54,54,54,54,54,54,54


In [13]:
mini_data_user = user_data[user_data["user_id:token"].isin(mini_data_u["user_id:token"])]
mini_data_user = mini_data_user.reset_index().reset_index().rename(columns={'level_0': 'user_id'})
mini_data_user.drop("index", axis=1, inplace=True)
mini_data_user

,user_id,user_id:token,user_name:token,user_review_count:float,yelping_since:float,user_useful:float,user_funny:float,user_cool:float,elite:token,fans:float,...,compliment_more:float,compliment_profile:float,compliment_cute:float,compliment_list:float,compliment_note:float,compliment_plain:float,compliment_cool:float,compliment_funny:float,compliment_writer:float,compliment_photos:float
0,0,-3s52C4zL_DHRK0ULG6qtg,Sara,1760,2010-05-16 16:21:33,1663,476,368,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2019",55,...,7,1,2,0,20,47,20,20,12,2
1,1,-7XrSrJfHndHc_taEXurTw,Paul,381,2014-07-04 12:27:52,557,155,263,"2015,2016,2017",8,...,0,0,0,0,6,10,19,19,19,6
2,2,-B-QEUESGWHPE_889WJaeg,Mark,1014,2009-05-31 06:02:42,3817,1689,2529,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2...",200,...,23,10,4,5,89,294,286,286,82,198
3,3,-FxsSuwDbIII7yo5BjHpiA,Tracy,793,2013-10-31 20:58:40,1734,608,1321,"2015,2016,2017,2018,2019,20,20,2021",48,...,6,8,1,0,53,101,126,126,49,79
4,4,-G7Zkl1wIWBBmD0KRy_sCw,Gerald,3131,2012-12-16 23:26:39,61899,31999,48327,"2013,2014,2015,2016,2017,2018,2019,20,20,2021",703,...,108,49,2,0,820,1478,1900,1900,1217,1529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582,1582,zsXoPyTcU8ThZGbtAB-Vug,Cora,180,2017-09-24 22:33:49,221,5,185,"2019,20,20,2021",8,...,9,1,0,0,10,4,13,13,6,5
1583,1583,zu-e06_BM_TdkAZEKMrIww,Majlinda,447,2012-09-25 02:58:07,949,185,420,"2013,2014,2015,2016,2017,2018",54,...,2,1,2,0,16,31,22,22,15,18
1584,1584,zv7tpu7xeaNyAeFG03d2CA,Sherese,214,2018-03-20 01:37:04,1102,293,701,"2019,20,20,2021",37,...,1,0,0,0,42,38,52,52,12,30
1585,1585,zwXmvn1op5LuFF2Kveqaug,Jake,188,2014-08-27 22:09:02,385,114,244,"2015,2016,2017,2018,2019",20,...,1,0,1,0,21,13,28,28,17,1


In [15]:
mini_data_in = inter_data[inter_data["user_id:token"].isin(mini_data_user["user_id:token"])]
mini_data_in

,user_id:token,item_id:token,rating:float,timestamp:float,useful:float,funny:float,cool:float,review_id:token
144,-3s52C4zL_DHRK0ULG6qtg,KWIapYSSHxfZ1NQFpV2S9g,1.0,1323279412,4,0,0,AO20GPrtj4arTImdmtUlgw
154,-3s52C4zL_DHRK0ULG6qtg,cljHYek6vHtqUgUsnJBqmA,1.0,1323466281,1,0,0,AlRlx9tpiKmPKiE-6I7Yxg
161,-3s52C4zL_DHRK0ULG6qtg,6Euj8bfFJ4aQ_cCi7aMWCw,1.0,1323467199,4,5,4,kNnAUWJufaBD1ylXLjPbhw
143,-3s52C4zL_DHRK0ULG6qtg,1MeIwdbTnZOBFCKOrgaxuw,1.0,1361042188,2,0,0,XuByFsXXhnKmZ6hxucOj7A
141,-3s52C4zL_DHRK0ULG6qtg,PrF1qZ9L9gQJKZkruB1dCg,1.0,1395423411,1,1,0,1pPnJuLtURzy0cFPIHprEQ
...,...,...,...,...,...,...,...,...
319288,zyvxtbh5eJ86bVgk52Yflg,q2rR-v9ZStix89t7ihyuqg,5.0,1573027532,1,0,2,gIidI9si68nr-hReal6WrA
319350,zyvxtbh5eJ86bVgk52Yflg,RcsTutp1103SGiYtam7xRg,5.0,1577719484,0,0,0,NaBQNBAE329_bTx6qgFWCA
319339,zyvxtbh5eJ86bVgk52Yflg,vD2jzpPv4iyOLKzITscGvA,5.0,1594643253,1,0,0,QHeAEaNYYY68tGXzrzSzbg
319346,zyvxtbh5eJ86bVgk52Yflg,RVJTAYG1RGzovNxhDabs9Q,5.0,1601030217,0,0,0,VLT90MbxpdWfUvfqMSXaAw


In [17]:
data_i = mini_data_in
data_i = data_i.groupby(['item_id:token']).count()
data_i["count"] = data_i["user_id:token"]
mini_data_i = data_i.reset_index()
mini_data_i

,item_id:token,user_id:token,rating:float,timestamp:float,useful:float,funny:float,cool:float,review_id:token,count
0,--N9yp3ZWqQIm7DqKRvorg,2,2,2,2,2,2,2,2
1,--O3ip9NpXTKD4oBS1pY2A,2,2,2,2,2,2,2,2
2,--S43ruInmIsGrnnkmavRw,7,7,7,7,7,7,7,7
3,--ZVrH2X2QXBFdCilbirsw,8,8,8,8,8,8,8,8
4,--sXnWH9Xm6_NvIjyuA99w,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...
16495,zyU0TtQqoA-hn5IwAWpP3A,1,1,1,1,1,1,1,1
16496,zymV2vdJfHNH63UQ7aUFYw,1,1,1,1,1,1,1,1
16497,zyrhpLocbo60EbS57jBTLw,7,7,7,7,7,7,7,7
16498,zzbZtgPYZS8sTIWQH6DwEw,13,13,13,13,13,13,13,13


In [19]:
mini_data_item = item_data[item_data["item_id:token"].isin(mini_data_i["item_id:token"])]
mini_data_item = mini_data_item.reset_index().reset_index().rename(columns={'level_0': 'item_id'})
mini_data_item.drop("index", axis=1, inplace=True)
mini_data_item

,item_id,item_id:token,item_name:token_seq,address:token_seq,city:token_seq,state:token,postal_code:token,latitude:float,longitude:float,item_stars:float,item_review_count:float,is_open:float,categories:token_seq
0,0,--N9yp3ZWqQIm7DqKRvorg,Firestone Complete Auto Care,6291 9th St N,St Petersburg,FL,33702,27.829290,-82.646870,2.5,23,1,"Automotive, Auto Repair, Auto Parts & Supplies..."
1,1,--O3ip9NpXTKD4oBS1pY2A,Alameda Park,1400 Santa Barbara St,Santa Barbara,CA,93101,34.428303,-119.705397,4.5,65,1,"Active Life, Parks"
2,2,--S43ruInmIsGrnnkmavRw,Peaches Records,4318 Magazine St,New Orleans,LA,70115,29.920622,-90.100819,3.5,91,1,"Music & DVDs, Vinyl Records, Fashion, Women's ..."
3,3,--ZVrH2X2QXBFdCilbirsw,Chris's Sandwich Shop,1531 W Wynnewood Rd,Ardmore,PA,19003,39.997299,-75.292207,4.5,32,0,"American (Traditional), Restaurants, Pizza, Sa..."
4,4,--sXnWH9Xm6_NvIjyuA99w,Philadelphia,NaN,Philadelphia,PA,NaN,39.952584,-75.165222,4.0,29,1,"Public Services & Government, Local Flavor"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16495,16495,zyU0TtQqoA-hn5IwAWpP3A,Auto Solutions,13809 N Dale Mabry Hwy,Tampa,FL,33618,28.072895,-82.506196,5.0,25,1,"Car Stereo Installation, Windshield Installati..."
16496,16496,zymV2vdJfHNH63UQ7aUFYw,"Imran Amir, MD - City Dermatology Skin Institute","3260 Tillman Dr, Ste 120",Bensalem,PA,19020,40.117203,-74.962614,3.0,21,1,"Health & Medical, Skin Care, Laser Hair Remova..."
16497,16497,zyrhpLocbo60EbS57jBTLw,Generations Hall,310 Andrew Higgins Dr,New Orleans,LA,70130,29.941828,-90.066328,3.5,28,1,"Venues & Event Spaces, Event Planning & Servic..."
16498,16498,zzbZtgPYZS8sTIWQH6DwEw,F & M Patio Bar,4841 Tchoupitoulas St,New Orleans,LA,70115,29.916857,-90.106974,3.0,86,1,"Food, American (Traditional), Nightlife, Resta..."


In [21]:
mini_data_inter = pd.merge(mini_data_in, mini_data_user[['user_id', 'user_id:token']], on='user_id:token', how='left')
mini_data_inter.drop("user_id:token", axis=1, inplace=True)
mini_data_user.drop("user_id:token", axis=1, inplace=True)
mini_data_inter

,item_id:token,rating:float,timestamp:float,useful:float,funny:float,cool:float,review_id:token,user_id
0,KWIapYSSHxfZ1NQFpV2S9g,1.0,1323279412,4,0,0,AO20GPrtj4arTImdmtUlgw,0
1,cljHYek6vHtqUgUsnJBqmA,1.0,1323466281,1,0,0,AlRlx9tpiKmPKiE-6I7Yxg,0
2,6Euj8bfFJ4aQ_cCi7aMWCw,1.0,1323467199,4,5,4,kNnAUWJufaBD1ylXLjPbhw,0
3,1MeIwdbTnZOBFCKOrgaxuw,1.0,1361042188,2,0,0,XuByFsXXhnKmZ6hxucOj7A,0
4,PrF1qZ9L9gQJKZkruB1dCg,1.0,1395423411,1,1,0,1pPnJuLtURzy0cFPIHprEQ,0
...,...,...,...,...,...,...,...,...
132594,q2rR-v9ZStix89t7ihyuqg,5.0,1573027532,1,0,2,gIidI9si68nr-hReal6WrA,1586
132595,RcsTutp1103SGiYtam7xRg,5.0,1577719484,0,0,0,NaBQNBAE329_bTx6qgFWCA,1586
132596,vD2jzpPv4iyOLKzITscGvA,5.0,1594643253,1,0,0,QHeAEaNYYY68tGXzrzSzbg,1586
132597,RVJTAYG1RGzovNxhDabs9Q,5.0,1601030217,0,0,0,VLT90MbxpdWfUvfqMSXaAw,1586


In [23]:
mini_data_inter = pd.merge(mini_data_inter, mini_data_item[['item_id', 'item_id:token']], on='item_id:token', how='left')
mini_data_inter.drop("item_id:token", axis=1, inplace=True)
mini_data_item.drop("item_id:token", axis=1, inplace=True)
mini_data_inter

,rating:float,timestamp:float,useful:float,funny:float,cool:float,review_id:token,user_id,item_id
0,1.0,1323279412,4,0,0,AO20GPrtj4arTImdmtUlgw,0,5497
1,1.0,1323466281,1,0,0,AlRlx9tpiKmPKiE-6I7Yxg,0,10517
2,1.0,1323467199,4,5,4,kNnAUWJufaBD1ylXLjPbhw,0,1840
3,1.0,1361042188,2,0,0,XuByFsXXhnKmZ6hxucOj7A,0,640
4,1.0,1395423411,1,1,0,1pPnJuLtURzy0cFPIHprEQ,0,6873
...,...,...,...,...,...,...,...,...
132594,5.0,1573027532,1,0,2,gIidI9si68nr-hReal6WrA,1586,13898
132595,5.0,1577719484,0,0,0,NaBQNBAE329_bTx6qgFWCA,1586,7337
132596,5.0,1594643253,1,0,0,QHeAEaNYYY68tGXzrzSzbg,1586,15261
132597,5.0,1601030217,0,0,0,VLT90MbxpdWfUvfqMSXaAw,1586,7304


In [26]:
new_mini_inter = mini_data_inter[["user_id","item_id","rating:float","timestamp:float"]]
new_mini_item = mini_data_item.drop(["city:token_seq","state:token","postal_code:token","latitude:float","longitude:float"],axis = 1)
new_mini_user = mini_data_user[["user_id","user_name:token","average_stars:float","user_review_count:float"]]


In [28]:
new_mini_inter

,user_id,item_id,rating:float,timestamp:float
0,0,5497,1.0,1323279412
1,0,10517,1.0,1323466281
2,0,1840,1.0,1323467199
3,0,640,1.0,1361042188
4,0,6873,1.0,1395423411
...,...,...,...,...
132594,1586,13898,5.0,1573027532
132595,1586,7337,5.0,1577719484
132596,1586,15261,5.0,1594643253
132597,1586,7304,5.0,1601030217


In [30]:
#########################   处理基本数据    ##########################################################

In [93]:
item_data = new_mini_item.sort_values(by=['item_id'], ascending=[True])
inter_data = mini_data_inter.sort_values(by=['user_id','rating:float', 'timestamp:float'], ascending=[True, False, True])
user_data = new_mini_user.sort_values(by=['user_id'], ascending=[True])

In [95]:
threshold = 3
max_length = 10
item_temp = "Item id: item_{item_id}, item title: {item_name}, address: {address}, item stars: {item_stars}, item review count: {item_review_count}, is open: {is_open}, liking_rate: {liking_rate}."
# user_temp = "User id: user_{user_id}, age: {age}, gender: {gender}, occupation: {occupation}, zipcode: {zip_code}, View: {most_viewed_genres}, Like: {best_rated_genres}."
user_temp = "User id: user_{user_id}, User name: {user_name}, Average stars: {average_stars}, User review count: {user_review_count}, Top View: {most_viewed_genres_top}, Like: {best_rated_genres}."

# # inter_temp = "The user user_{user_id} likes the following movies: {likes} and dislikes the following movies: {dislikes}."
user_train_temp = """You are a recommendation system that gives you a list of items that the user likes and dislikes, and predicts whether the user will like the target item or not. Returns only whether the user likes the target movie: Yes or No. # noqa: E501

User Information:
{user_info}

Likes List:
{likes_list}

Dislikes List:
{dislikes_list}

Target Movie:
{targe_movie}
"""

In [97]:
# 产品评价
def build_item_other(row):
    df1 = inter_data[inter_data['item_id'] == row['item_id']]
    total = df1.shape[0]
    likes_count = df1[df1['rating:float'] > threshold].shape[0]
    # dislikes_count = total - likes_count
    liking_rate = round(likes_count/total, 4)
    return {'liking_rate':liking_rate}

In [99]:
item_data['other'] = item_data.apply(lambda row: build_item_other(row), axis=1)


In [101]:
item_data

,item_id,item_name:token_seq,address:token_seq,item_stars:float,item_review_count:float,is_open:float,categories:token_seq,other
0,0,Firestone Complete Auto Care,6291 9th St N,2.5,23,1,"Automotive, Auto Repair, Auto Parts & Supplies...",{'liking_rate': 1.0}
1,1,Alameda Park,1400 Santa Barbara St,4.5,65,1,"Active Life, Parks",{'liking_rate': 1.0}
2,2,Peaches Records,4318 Magazine St,3.5,91,1,"Music & DVDs, Vinyl Records, Fashion, Women's ...",{'liking_rate': 1.0}
3,3,Chris's Sandwich Shop,1531 W Wynnewood Rd,4.5,32,0,"American (Traditional), Restaurants, Pizza, Sa...",{'liking_rate': 0.875}
4,4,Philadelphia,NaN,4.0,29,1,"Public Services & Government, Local Flavor",{'liking_rate': 1.0}
...,...,...,...,...,...,...,...,...
16495,16495,Auto Solutions,13809 N Dale Mabry Hwy,5.0,25,1,"Car Stereo Installation, Windshield Installati...",{'liking_rate': 1.0}
16496,16496,"Imran Amir, MD - City Dermatology Skin Institute","3260 Tillman Dr, Ste 120",3.0,21,1,"Health & Medical, Skin Care, Laser Hair Remova...",{'liking_rate': 0.0}
16497,16497,Generations Hall,310 Andrew Higgins Dr,3.5,28,1,"Venues & Event Spaces, Event Planning & Servic...",{'liking_rate': 0.7143}
16498,16498,F & M Patio Bar,4841 Tchoupitoulas St,3.0,86,1,"Food, American (Traditional), Nightlife, Resta...",{'liking_rate': 0.3077}


In [103]:
item_data["category_list"] = item_data["categories:token_seq"].str.split(',')
item_data["category_list"]

0        [Automotive,  Auto Repair,  Auto Parts & Suppl...
1                                    [Active Life,  Parks]
2        [Music & DVDs,  Vinyl Records,  Fashion,  Wome...
3        [American (Traditional),  Restaurants,  Pizza,...
4            [Public Services & Government,  Local Flavor]
                               ...                        
16495    [Car Stereo Installation,  Windshield Installa...
16496    [Health & Medical,  Skin Care,  Laser Hair Rem...
16497    [Venues & Event Spaces,  Event Planning & Serv...
16498    [Food,  American (Traditional),  Nightlife,  R...
16499         [Food,  Restaurants,  Coffee & Tea,  Donuts]
Name: category_list, Length: 16500, dtype: object

In [105]:
# 拼接信息, json
item_data['info'] = item_data.apply(lambda row: {'text':item_temp.replace('{item_name}',row["item_name:token_seq"]).replace('{item_id}',str(row["item_id"])).replace('{address}',str(row["address:token_seq"])).replace('{item_stars}',str(row["item_stars:float"])).replace('{item_review_count}',str(row["item_review_count:float"])).replace('{is_open}',str(row["is_open:float"])).replace('{class}',', '.join(map(str, row["category_list"]))).replace('{liking_rate}',str(row['other']["liking_rate"]))}, axis=1)

In [107]:
item_data['info']

0        {'text': 'Item id: item_0, item title: Firesto...
1        {'text': 'Item id: item_1, item title: Alameda...
2        {'text': 'Item id: item_2, item title: Peaches...
3        {'text': 'Item id: item_3, item title: Chris's...
4        {'text': 'Item id: item_4, item title: Philade...
                               ...                        
16495    {'text': 'Item id: item_16495, item title: Aut...
16496    {'text': 'Item id: item_16496, item title: Imr...
16497    {'text': 'Item id: item_16497, item title: Gen...
16498    {'text': 'Item id: item_16498, item title: F &...
16499    {'text': 'Item id: item_16499, item title: Dun...
Name: info, Length: 16500, dtype: object

In [109]:
# 分组&聚合
data_new = inter_data
data_u = data_new.groupby(['user_id']).count()
data_u["count"] = data_u["item_id"]
# data_u = data_u.drop(['item_id:token','num_repeat:float','timestamp:float'], axis=1)

# 筛选=>10 core
data_u = data_u[data_u["count"] >= 10]

# 按user_id筛选
inter_data = data_new[data_new["user_id"].isin(data_u.index)]
# 按user_id分组，并将其他字段存为列表
inter_data = inter_data.groupby('user_id').agg(list).reset_index()
user_data

,user_id,user_name:token,average_stars:float,user_review_count:float
0,0,Sara,3.46,1760
1,1,Paul,4.34,381
2,2,Mark,3.41,1014
3,3,Tracy,4.15,793
4,4,Gerald,3.68,3131
...,...,...,...,...
1582,1582,Cora,4.49,180
1583,1583,Majlinda,3.84,447
1584,1584,Sherese,4.08,214
1585,1585,Jake,4.33,188


In [111]:
# 用户使用的产品类型列表
# inter_data.iloc[0]
# inter_data.iloc[0]['item_id:token']
# 将嵌套列表展开为一个平面列表
# category_list = [category for item_id in inter_data.iloc[0]['item_id:token'] for category in category_mapping[item_data.loc[item_data['item_id:token'] == item_id,'class:token_seq'].values[0]]]
# df = pd.DataFrame(category_list, columns=['category'])
# 统计各个值出现的次数
#counts = df['category'].value_counts().reset_index()
# print(counts)

item_to_category = dict(zip(item_data['item_id'],item_data['category_list']))

def calculate_category_count_top(items):
    categories = [cat for item in items['item_id'] for cat in item_to_category.get(item, [])]
    df = pd.DataFrame(categories, columns=['category'])
    count = df['category'].value_counts().head(5).reset_index().to_dict('records')
    res = {item['category']: item['count'] for item in count}
    return res
    
def calculate_category_count(items):
    categories = [cat for item in items['item_id'] for cat in item_to_category.get(item, [])]
    return pd.Series(categories).value_counts().to_dict()
    
user_data['most_viewed_genres'] = inter_data.apply(lambda row: calculate_category_count(row),axis = 1)
user_data['most_viewed_genres_top'] = inter_data.apply(lambda row: calculate_category_count_top(row),axis = 1)


In [113]:
user_data['most_viewed_genres_top'].iloc[0]

{' Restaurants': 49,
 ' Food': 31,
 ' Breakfast & Brunch': 23,
 ' American (New)': 18,
 ' Coffee & Tea': 17}

In [115]:
def calculate_category_review(items):
    df1 = items
    likes_count = len([x for x in items['rating:float'] if x > 3])
    categories = [cat for item in items['item_id'][0:likes_count] for cat in item_to_category.get(item, [])]
    df = pd.DataFrame(categories, columns=['category'])
    count = df['category'].value_counts().head(5).reset_index().to_dict('records')
    res = {item['category']: item['count'] for item in count}
    return res


user_data['best_rated_genres'] = inter_data.apply(lambda row: calculate_category_review(row),axis = 1)

In [116]:
user_data['best_rated_genres'].iloc[0]

{' Restaurants': 23,
 ' Food': 13,
 ' American (New)': 13,
 ' Nightlife': 11,
 ' Bars': 10}

In [119]:
def esteem(row):
    res =  {key: row['best_rated_genres'][key] / row['most_viewed_genres'][key] for key in row['best_rated_genres']} 
    return res


user_data['esteem'] = user_data.apply(lambda row: esteem(row), axis = 1)

In [121]:
user_data['esteem']

0       {' Restaurants': 0.46938775510204084, ' Food':...
1       {' Restaurants': 0.7878787878787878, ' Food': ...
2       {' Restaurants': 0.4827586206896552, ' Bars': ...
3       {' Restaurants': 0.7323943661971831, ' Food': ...
4       {' Restaurants': 0.6036866359447005, 'Restaura...
                              ...                        
1582    {' Restaurants': 0.8888888888888888, ' Food': ...
1583    {' Restaurants': 0.7441860465116279, ' Food': ...
1584    {' Food': 0.8947368421052632, ' Shopping': 0.8...
1585    {' Restaurants': 0.7105263157894737, ' Nightli...
1586    {' Restaurants': 0.7297297297297297, ' Bars': ...
Name: esteem, Length: 1587, dtype: object

In [123]:
def cal_arr(ls,num):
    arr = []
    for i in range(0, len(ls), num):
        a1 = ls[i:i+num]
        if len(a1)==num:
            arr.append(a1)
    return arr
def build_user_inter(row):
    rs = row['rating:float']
    p_len = len([x for x in rs if x > threshold])
    p_rs = rs[:p_len]
    n_rs = rs[p_len:]

    ids = row['item_id']
    p_id_groups = ids[:p_len]
    n_id_groups = ids[p_len:]

    # 6p,4n
    p_num,n_num = 6,4
    p_rs1 = cal_arr(p_rs,p_num)
    n_rs1 = cal_arr(n_rs,n_num)

    p_id_groups1 = cal_arr(p_id_groups,p_num)
    n_id_groups1 = cal_arr(n_id_groups,n_num)
    
    # 对齐两个列表
    min_num = min(len(p_rs1), len(n_rs1))
    p_rs = p_rs1[:min_num]
    n_rs = n_rs1[:min_num]

    p_id_groups = p_id_groups1[:min_num]
    n_id_groups = n_id_groups1[:min_num]

    inter = {'p_id_groups': p_id_groups, 'n_id_groups': n_id_groups, 'p_rs': p_rs, 'n_rs': n_rs}
    return inter

inter_data['user_inter'] = inter_data.apply(lambda row: build_user_inter(row), axis=1)
inter_data.iloc[0]

user_id                                                            0
rating:float       [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...
timestamp:float    [1287647191, 1296899780, 1303648349, 132820974...
useful:float       [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 3, 1, ...
funny:float        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...
cool:float         [0, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...
review_id:token    [71loE4umBr6MA_FoOwgQ6A, vjxpnpAzofSF5vYh9UaTo...
item_id            [13897, 388, 13897, 13897, 12437, 12113, 1313,...
user_inter         {'p_id_groups': [[13897, 388, 13897, 13897, 12...
Name: 0, dtype: object

In [125]:
inter_data['user_inter'].iloc[0]


{'p_id_groups': [[13897, 388, 13897, 13897, 12437, 12113],
  [1313, 9938, 4892, 3499, 6971, 4702],
  [9073, 14410, 16138, 4024, 15261, 11196],
  [13784, 2407, 2072, 15368, 685, 1990],
  [12189, 419, 14156, 419, 8336, 13580],
  [2613, 4136, 5785, 9577, 11094, 9244],
  [10223, 13220, 14156, 6537, 9624, 6467]],
 'n_id_groups': [[10036, 4348, 4024, 7139],
  [8336, 6735, 6376, 12844],
  [8692, 7446, 2027, 4566],
  [419, 10968, 12402, 8805],
  [12036, 11325, 4566, 8336],
  [13702, 7468, 11571, 11326],
  [11486, 640, 3343, 5609]],
 'p_rs': [[5.0, 5.0, 5.0, 5.0, 5.0, 5.0],
  [5.0, 5.0, 5.0, 5.0, 5.0, 5.0],
  [5.0, 5.0, 4.0, 4.0, 4.0, 4.0],
  [4.0, 4.0, 4.0, 4.0, 4.0, 4.0],
  [4.0, 4.0, 4.0, 4.0, 4.0, 4.0],
  [4.0, 4.0, 4.0, 4.0, 4.0, 4.0],
  [4.0, 4.0, 4.0, 4.0, 4.0, 4.0]],
 'n_rs': [[3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 2.0, 2.0],
  [2.0, 2.0, 2.0, 2.0],
  [2.0, 2.0, 2.0, 2.0]]}

In [127]:
user_data['info'] = user_data.apply(lambda row: {'text':user_temp.replace('{user_id}',str(row["user_id"])).replace('{user_name}',str(row["user_name:token"])).replace('{average_stars}',str(row["average_stars:float"])).replace('{user_review_count}',str(row["user_review_count:float"])).replace('{most_viewed_genres_top}',str(row["most_viewed_genres_top"])).replace('{most_viewed_genres}',str(row["most_viewed_genres_top"])).replace('{best_rated_genres}',str(row["esteem"]))}, axis=1)
user_data['info'].iloc[0]

{'text': "User id: user_0, User name: Sara, Average stars: 3.46, User review count: 1760, Top View: {' Restaurants': 49, ' Food': 31, ' Breakfast & Brunch': 23, ' American (New)': 18, ' Coffee & Tea': 17}, Like: {' Restaurants': 0.46938775510204084, ' Food': 0.41935483870967744, ' American (New)': 0.7222222222222222, ' Nightlife': 0.6875, ' Bars': 0.6666666666666666}."}

In [129]:
def cal_label(rating):
    return "Yes" if rating > threshold else "No"

In [131]:
def build_item_infos(ids):
    item_infos = ''
    for i in ids:
        # item_row = item_data[item_data["item_id:token"]==i].iloc[0]
        item_infos += f'item id: item_{i} \n'
    return item_infos

def build_inter(row):
    #拼接用户信息
    user_row = user_data[user_data["user_id"]==row["user_id"]].iloc[0]
    # user_info = f'user id: user_{user_row["user_id:token"]}, age: {user_row["age:token"]}, gender: {user_row["gender:token"]}, occupation: {user_row["occupation:token"]}, zipcode: {user_row["zip_code:token"]}, View: {user_row["most_viewed_genres:object"]}, Like: {user_row["best_rated_genres:object"]}'
    user_info = f'user id: user_{user_row["user_id"]}, user name: {user_row["user_name:token"]}, average stars: {user_row["average_stars:float"]}, user review count: {user_row["user_review_count:float"]}, View: {user_row["most_viewed_genres_top"]}, Like: {user_row["esteem"]}'
    
    #细分输入数据，按照6：4比率划分，且随机取出一个数据作为target
    user_inter = row['user_inter']
    p_id_groups = user_inter['p_id_groups']
    n_id_groups = user_inter['n_id_groups']
    if random.random() < 0.8:
        add = 'ft'
        ft_user_list.append(row['user_id'])
    else:
        add = 'pd'
        pd_user_list.append(row['user_id'])
    for p_ids,n_ids in zip(p_id_groups,n_id_groups):
        # [0,1)根据0.6和0.4的比例进行线性转换
        if random.random() < 0.6:
            iid = p_ids.pop()
            flag = 'Yes'
        else:
            iid = n_ids.pop()
            flag = 'No'
        target_item_row = item_data[item_data["item_id"]==iid].iloc[0]
        target_movie = f'item id: item_{iid}'
        #拼接prompt
        user_temp = user_train_temp.replace('{user_info}', user_info).replace('{targe_movie}',target_movie).replace('{likes_list}', build_item_infos(p_ids)).replace('{dislikes_list}', build_item_infos(n_ids)).replace('\n\n\n','\n\n')
        user_dict = {"instruction": user_temp, "input": '', "output": flag}
        if add == 'ft':
            ft_list.append(user_dict)
        else:
            pd_list.append(user_dict)

In [133]:
# 数据分组   微调、预测组，预测组用户从微调组中取出
ft_list = [] #微调
pd_list = [] #预测
ft_user_list = [] #微调用户列表
pd_user_list = [] #预测用户列表
for idx, row in inter_data.iterrows():
    build_inter(row)

In [134]:
print(len(ft_user_list))
print(len(ft_list))
print(len(pd_user_list))
print(len(pd_list))

1299
6681
288
1470


In [137]:
ft_list[10]

{'instruction': "You are a recommendation system that gives you a list of items that the user likes and dislikes, and predicts whether the user will like the target item or not. Returns only whether the user likes the target movie: Yes or No. # noqa: E501\n\nUser Information:\nuser id: user_2, user name: Mark, average stars: 3.41, user review count: 1014, View: {' Restaurants': 58, 'Restaurants': 26, ' Bars': 24, ' Nightlife': 23, ' Food': 19}, Like: {' Restaurants': 0.4827586206896552, ' Bars': 0.5416666666666666, ' Nightlife': 0.5652173913043478, ' American (New)': 0.5882352941176471, 'Restaurants': 0.34615384615384615}\n\nLikes List:\nitem id: item_15062 \nitem id: item_6787 \nitem id: item_9635 \nitem id: item_10427 \nitem id: item_6470 \n\nDislikes List:\nitem id: item_15698 \nitem id: item_11015 \nitem id: item_305 \nitem id: item_5912 \n\nTarget Movie:\nitem id: item_2471\n",
 'input': '',
 'output': 'Yes'}

In [139]:
# list类型数据转化为json
def export_json(data,export_path):
    df = pd.DataFrame(data)
    # 导出为格式化后的 JSON 文件
    df.to_json(export_path, orient='records')

In [141]:
#调用写入文件中
export_json(item_data['info'].tolist()+user_data['info'].tolist(),w_pt_path)
export_json(pd_list,w_pd_path)
export_json(ft_list,w_ft_path)

In [143]:
# 读取 JSON 文件到 DataFrame
text_df = pd.read_json(w_pt_path)
text_df

,text
0,"Item id: item_0, item title: Firestone Complet..."
1,"Item id: item_1, item title: Alameda Park, add..."
2,"Item id: item_2, item title: Peaches Records, ..."
3,"Item id: item_3, item title: Chris's Sandwich ..."
4,"Item id: item_4, item title: Philadelphia, add..."
...,...
18082,"User id: user_1582, User name: Cora, Average s..."
18083,"User id: user_1583, User name: Majlinda, Avera..."
18084,"User id: user_1584, User name: Sherese, Averag..."
18085,"User id: user_1585, User name: Jake, Average s..."
